In [2]:
import random

In [3]:
def ParsePath(paths):
    good = []
    bad = []
    for path in paths:
        path_2 = path.split('/')
        if int(path_2[2])%2 == 1:
            if '_main' not in path_2[-1]:
                bad.append(path)
        else:
            good.append(path)
    return bad, good

In [4]:
file_list = open('index.txt', 'r').read().split('\n')[:-1]
bad = ParsePath(file_list)

Data Cleaning Functions

In [5]:
def findline(lines, target_string, num_line = 0):
    # locate all lines that contains a keyword "target_string", recursively.
    if num_line == len(lines):
        return -1
    if target_string in lines[num_line]:
        return num_line
    else:
        return findline(lines, target_string, num_line + 1)
    
def FindAllFunctions(lines, file_name, function_starts = [], func_ns = [], func_number = 1, num_line = 0):
    #Recursively find all functions in the file
    if num_line == len(lines) - 1:
        return function_starts, func_ns
    else:
        num_line += 1
        func_n = format(func_number, '03d')
        if ' ' + file_name + '_' + func_n + ' (' in lines[num_line] or ' ' + file_name + '_' + func_n + '(' in lines[num_line]:
            function_starts.append(num_line)
            func_ns.append(func_n)
            func_number += 1
        return FindAllFunctions(lines, file_name, function_starts, func_ns, func_number, num_line)
    
def FindCommentBlocks(lines, comment_starts = [], comment_ends = [], num_line = 0):
    if num_line == len(lines) - 1:
        return comment_starts, comment_ends
    else:
        num_line += 1
        if lines[num_line] == '/*':
            comment_starts.append(num_line)
        elif lines[num_line] == '*/':
            comment_ends.append(num_line)
        return FindCommentBlocks(lines, comment_starts, comment_ends, num_line)

def deletecomment(string):
    if '/*' not in string:
        return string
    comment_start = string.find('/*')
    comment_end = string.find('*/')
    return deletecomment(string[:comment_start] + string[comment_end + 2:])

def cleanupfunction(function):
    #Delete redundant spaces and tabs
    function0 = [line.lstrip().rstrip() for line in function]
    function = []
    
    #Delete compiler instruction lines
    for line in function0:
        if len(line) > 0:
            if line[0] != '#':
                function.append(line)
    function = ''.join(function)
    signs = ['=', '+', '-', '/', '!=', '==', '<<', '>>', ',', ')', '(', '}', '{']
    
    # Delete redundant spaces before and after a syntax
    for sign in signs:
        function = function.replace(' ' + sign, sign)
        function = function.replace(sign + ' ', sign)
    function = function.replace('if ', 'if')
    return function
    
def ParseFile(file_name, good_or_bad):
    file = open(file_name, 'r')
    f = file.read().split('\n')
    f = [line.lstrip().rstrip() for line in f]
    file.close()
    f_name = file_name.split('/')[-1].split('.')[0]
    function_starts, func_ns = FindAllFunctions(f, f_name, function_starts = [], func_ns = [], func_number = 1, num_line = 0)
    comment_starts, comment_ends = FindCommentBlocks(f, comment_starts = [], comment_ends = [], num_line = 0)
    comment_starts = comment_starts[2:]
    comment_ends = comment_ends[1: -1]
    
    function_list = []
    
    for i in range(len(function_starts)):
        function = f[comment_ends[i]+1:comment_starts[i]]
        function = cleanupfunction(function)
        function = deletecomment(function)
        function = function.replace(f_name + '_', '')
        if good_or_bad:
            function = function.replace(func_ns[i], 'good')
        else:
            function = function.replace(func_ns[i], 'bad')
        
        while '  ' in function:
            function = function.replace('  ', ' ')
        function_list.append(function)
        
    return function_list, f_name

Parsing code instance into list of syntax.

In [11]:
def toWordLevel(instance):
    instance = ' '.join(instance.split())
    parser_list_lv1 = ['==', '!=', '&&', '||', '<=', '>=', '>>', '<<']
    parser_list_lv2 = ['!', ';', '=', '+', '-', '&', '%', '*', ':', '.', '|', '/', '(', ')', '{', '}', '[', ']', '<', '>', '\'', '\"', ',', '_', ' ']
    
    parselv1 = []
    while len(instance) > 2:
        i = 0
        while True:
            if instance[i:i+2] in parser_list_lv1:
                if i != 0:
                    parselv1.append(instance[:i])
                parselv1.append(instance[i:i+2])
                instance = instance[i+2:]
                break
            if i == len(instance):
                parselv1.append(instance)
                instance = ''
                break
            i += 1
    parselv2 = []
    for st in parselv1:
        if st not in parser_list_lv1:
            while len(st) > 0:
                i = 0
                while True:
                    if i == len(st):
                        parselv2.append(st)
                        st = ''
                        break
                    if st[i] in parser_list_lv2:
                        if i != 0:
                            parselv2.append(st[:i])
                        parselv2.append(st[i])
                        st = st[i+1:]
                        break
                    i += 1
        else:
            parselv2.append(st)
    return parselv2

In [12]:
data_file = open('data.txt', 'w+')
src_train = open('src_train.txt', 'w+')
src_test = open('src_test.txt', 'w+')
src_val = open('src_val.txt', 'w+')
count = 0

for i in range(len(bad)):
    #array 'bad' contains the function names 
    bad_fs, _= ParseFile(bad[i], good_or_bad = False)
    for j in range(len(bad_fs)):
        ins = toWordLevel(bad_fs[j])
        if len(ins) < 400:
            rand = random.random()
            count += 1
            print(format(i, '03d'), bad_fs[j], file = data_file)
            if rand < 0.80:
                print(format(i, '03d'), bad_fs[j], file = src_train)
            elif rand >=0.80 and rand < 0.90: 
                print(format(i, '03d'), bad_fs[j], file = src_test)
            else:
                print(format(i, '03d'), bad_fs[j], file = src_val)
print(count)
data_file.close()
src_train.close()
src_test.close()
src_val.close()

580


Since This is transfer Learning, we need to include new vocabulary that did not appear in the old dataset.

In [13]:
total = open('data.txt', 'r').read().split('\n')[:-1]
total = [total[i][4:] for i in range(len(total))]
vocab = {}
vocab_file = open('vocab_new.txt', 'w+')
for ins in total:
    lst = toWordLevel(ins)
    for word in lst:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1
for word in vocab.keys():
    print(word, file = vocab_file)
vocab_file.close()
vocab_old = open('vocab_old.txt', 'r').read().split('\n')
vocab_new = open('vocab_new.txt', 'r').read().split()
vocab_tt = open('vocab_combined.txt', 'w+')
vocab_total = [word for word in vocab_old]

for word in vocab_new:
    if word not in vocab_old:
        vocab_total.append(word)
for word in vocab_total:
    print(word, file = vocab_tt)
vocab_tt.close()